<a href="https://colab.research.google.com/github/DannyZid/custom-seo-crawler/blob/main/scrapy_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Scrapy in Colab
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.2 MB/s eta 0:00:00


In [2]:
import scrapy
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule
from datetime import datetime
from scrapy.crawler import CrawlerProcess
import pandas as pd
import time

class MyCrawlSpider(CrawlSpider):
    name = 'SEOcrawlspider'
    allowed_domains = ['http-statuscode.com']
    start_urls = ['https://http-statuscode.com/errorCodeExample.php?code=429']

    rules = (
        Rule(LinkExtractor(allow=None), callback='parse_item', follow=True),
    )

    custom_settings = {
        'RETRY_TIMES': 3,
        'RETRY_HTTP_CODES': [429],
        'USER_AGENT': 'SEO_BZ_Spider',
        'FEED_FORMAT': 'jsonlines',
        'FEED_URI': '/content/myproject/output.json',
        'LOG_LEVEL': 'INFO',
    }

    def parse(self, response):
        url = response.url
        crawldate = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        title = response.css('title::text').get()
        meta_description = response.css('meta[name="description"]::attr(content)').get()
        h1_tag = response.css('h1::text').get()
        canonical_tag = response.css('link[rel="canonical"]::attr(href)').get()
        meta_robots_tag = response.css('meta[name="robots"]::attr(content)').get()
        server_response = response.status

        # Retry logic with delay for 429 status code
        if response.status == 429:
            time.sleep(90)
            yield scrapy.Request(url, callback=self.parse, dont_filter=True)
        else:
            yield {
                'url': url,
                'crawldate': crawldate,
                'title': title,
                'meta_description': meta_description,
                'h1_tag': h1_tag,
                'canonical_tag': canonical_tag,
                'meta_robots_tag': meta_robots_tag,
                'server_response': server_response,
            }

process = CrawlerProcess()

process.crawl(MyCrawlSpider)
process.start()

# Load the results into a Pandas DataFrame
result_df = pd.read_json('/content/myproject/output.json', lines=True)

# Display the DataFrame
result_df


INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
2023-12-13 14:00:21 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-5.15.120+-x86_64-with-glibc2.35
2023-12-13 14:00:21 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2023-12-13 14:00:21 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_F

""
